### Import Libraries

In [1]:
#import necessary libraries
import gc
import pandas as pd
pd.set_option('display.max_columns', None)

import numpy as np
from pandas import pivot
from pandas import Timestamp

import warnings
warnings.filterwarnings("ignore")

import findspark
findspark.init()

import pyspark
import databricks.koalas as ks
from pyspark.sql import SparkSession

from pyspark.sql.functions import *
from pyspark.sql.types import *

### Define Spark Configuration

In [2]:
spark = SparkSession.builder \
                    .appName('SparkODBC') \
                    .master('local[*]') \
                    .config('spark.sql.execution.arrow.pyspark.enabled', True) \
                    .config('spark.sql.session.timeZone', 'UTC') \
                    .config('spark.driver.memory','32G') \
                    .config('spark.ui.showConsoleProgress', True) \
                    .config('spark.sql.repl.eagerEval.enabled', True) \
                    .getOrCreate()

In [3]:
spark

### Define Schema - based on the actual table

In [4]:
HraRecords_schema = StructType([StructField("Age", StringType(), True),\
                                StructField("Attrition", StringType(), True),\
                                StructField("BusinessTravel", StringType(), True),\
                                StructField("DailyRate", FloatType(), True),\
                                StructField("Department", StringType(), True),\
                                StructField("DistanceFromHome", StringType(), True),\
                                StructField("Education", StringType(), True),\
                                StructField("EducationField", StringType(), True),\
                                StructField("EmployeeCount", StringType(), True),\
                                StructField("EmployeeNumber", StringType(), True),\
                                StructField("EnvironmentSatisfaction", StringType(), True),\
                                StructField("Gender", StringType(), True),\
                                StructField("HourlyRate", FloatType(), True),\
                                StructField("JobInvolvement", StringType(), True),\
                                StructField("JobLevel", StringType(), True),\
                                StructField("JobRole", StringType(), True),\
                                StructField("JobSatisfaction", StringType(), True),\
                                StructField("MaritalStatus", StringType(), True),\
                                StructField("MonthlyIncome", FloatType(), True),\
                                StructField("MonthlyRate", FloatType(), True),\
                                StructField("NumCompaniesWorked", StringType(), True),\
                                StructField("Over18", StringType(), True),\
                                StructField("OverTime", StringType(), True),\
                                StructField("PercentSalaryHike", FloatType(), True),\
                                StructField("PerformanceRating", StringType(), True),\
                                StructField("RelationshipSatisfaction", StringType(), True),\
                                StructField("StandardHours", StringType(), True),\
                                StructField("StockOptionLevel", StringType(), True),\
                                StructField("TotalWorkingYears", StringType(), True),\
                                StructField("TrainingTimesLastYear", StringType(), True),\
                                StructField("WorkLifeBalance", StringType(), True),\
                                StructField("YearsAtCompany", StringType(), True),\
                                StructField("YearsInCurrentRole", StringType(), True),\
                                StructField("YearsSinceLastPromotion", StringType(), True),\
                                StructField("YearsWithCurrManager", StringType(), True)])


### Define - Driver details in PyODBC

In [5]:
import pandas as pd
import pyodbc 

conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=LAPTOP-N2QTQTD1;'
                      'Database=EMP_DETAILS;'
                      'Trusted_Connection=yes;')

### Define the Query to be run

In [6]:
query = '''SELECT * FROM [EMP_DETAILS].[dbo].[HRA_Records]'''

### Download data from SQL Server in chunks of 1 Million and save the results to a Spakr Dataframe as shown below

In [7]:
#########################################################################################

dfs = []
for chunk in pd.read_sql_query(query, con=conn, chunksize=1000000):
    dfs.append(chunk)
    
HRA_Records = spark.createDataFrame(pd.concat(dfs), schema=HraRecords_schema)

#########################################################################################

In [8]:
HRA_Records

Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,MonthlyRate,NumCompaniesWorked,Over18,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
42,Yes,Travel_Frequently,130.0,Hardware,35,4,Other,1,903,3,Male,91.0,1,4,Sales Representative,4,Single,10890.0,43560.0,1,Y,Yes,36.0,1,3,80,3,30,2,4,9,9,2,3
47,No,Non-Travel,460.0,Sales,39,3,Other,1,904,3,Male,108.0,2,5,Healthcare Repres...,3,Single,40869.0,899118.0,4,Y,No,3.0,4,4,80,3,30,2,1,14,9,13,2
28,Yes,Travel_Rarely,430.0,Support,20,5,Life Sciences,1,905,3,Male,144.0,1,1,Sales Representative,4,Married,14550.0,421950.0,4,Y,Yes,43.0,2,1,80,2,7,1,2,4,4,1,3
28,Yes,Travel_Frequently,1486.0,Human Resources,5,1,Human Resources,1,906,4,Male,176.0,1,3,Sales Representative,3,Single,12777.0,63885.0,4,Y,Yes,25.0,4,4,80,2,31,5,2,5,4,4,1
28,Yes,Non-Travel,966.0,Human Resources,30,5,Medical,1,907,1,Male,200.0,2,2,Research Scientist,4,Divorced,23833.0,643491.0,0,Y,Yes,34.0,1,4,80,4,29,5,4,17,15,4,16
31,No,Non-Travel,1129.0,Human Resources,21,4,Human Resources,1,908,4,Male,116.0,4,4,Research Director,3,Married,37259.0,37259.0,1,Y,Yes,4.0,2,1,80,3,11,4,4,1,1,1,1
32,No,Non-Travel,601.0,Research & Develo...,32,3,Technical Degree,1,909,2,Male,95.0,2,1,Healthcare Repres...,4,Married,27389.0,520391.0,8,Y,No,21.0,3,4,80,3,11,4,4,4,3,4,4
38,No,Non-Travel,205.0,Research & Develo...,42,1,Other,1,910,4,Male,127.0,4,4,Manager,2,Divorced,5291.0,153439.0,6,Y,Yes,33.0,4,4,80,1,32,4,1,5,2,4,4
59,No,Non-Travel,998.0,Human Resources,46,5,Other,1,911,2,Female,35.0,3,2,Developer,2,Married,37358.0,410938.0,5,Y,Yes,27.0,4,4,80,1,17,3,2,14,4,10,8
29,Yes,Non-Travel,143.0,Sales,45,1,Marketing,1,912,2,Female,37.0,4,4,Healthcare Repres...,3,Divorced,11865.0,201705.0,2,Y,Yes,42.0,3,4,80,4,33,5,3,6,1,3,5


### Drop Duplicates if any - from HRA_Records dataframe

In [10]:
HRA_Records = HRA_Records.dropDuplicates()

### Create a virtual table by the name "HRA_Records"

In [11]:
HRA_Records.createOrReplaceTempView("HRA_Records")

### The below query shows the total counts of "Attrition". Although there is no huge difference, but overall counts of Attrition is less 

In [13]:
Attrition_Counts = spark.sql('''SELECT Attrition, COUNT(*) AS Total_Counts FROM HRA_Records
                                GROUP BY Attrition
                                ORDER BY Attrition''')

In [14]:
Attrition_Counts

Attrition,Total_Counts
No,2500474
Yes,2499526


### Attrition based on Departments - The below query shows that employees belonging to "Support" and "Sales" Department had higher attrition rate where "Support" team was dominant

In [30]:
Attrition_Dept_Counts = spark.sql('''SELECT Department, COUNT(*) AS Total_Counts FROM HRA_Records
                                     WHERE Attrition = "Yes"
                                     GROUP BY Department
                                     ORDER BY Total_Counts DESC''')

In [31]:
Attrition_Dept_Counts

Department,Total_Counts
Support,417150
Sales,417098
Software,416729
Research & Develo...,416506
Human Resources,416394
Hardware,415649


### We can see from the below query that "Male" have a higher attrition rate than the "Females" - in this dataset

In [48]:
Attrition_Gender = spark.sql('''SELECT Gender, COUNT(*) AS Total_Counts FROM HRA_Records
                                WHERE Attrition = "Yes"
                                GROUP BY Gender
                                ORDER BY Total_Counts''')

In [49]:
Attrition_Gender

Gender,Total_Counts
Female,1248189
Male,1251337


### Does personal life pave way for cossing career? In this case it sure does - Attrition rates is higest amongst the "Divorced" followed by "Single / Unmarried"

In [50]:
Attrition_MaritalStatus = spark.sql('''SELECT MaritalStatus, COUNT(*) AS Total_Counts FROM HRA_Records
                                       WHERE Attrition = "Yes"
                                       GROUP BY MaritalStatus
                                       ORDER BY Total_Counts DESC''')

In [51]:
Attrition_MaritalStatus

MaritalStatus,Total_Counts
Divorced,833908
Single,833761
Married,831857


### Amongst the given job roles, the Job Satisfaction appears to be highest among "Research Director", "Sales Representative" and "Developer"

In [52]:
Attrition_JobRole = spark.sql('''SELECT JobRole, ROUND(AVG(JobSatisfaction),0) AS Total_Avg FROM HRA_Records
                                 WHERE Attrition = "Yes"
                                 GROUP BY JobRole
                                 ORDER BY Total_Avg DESC''')

In [53]:
Attrition_JobRole

JobRole,Total_Avg
Research Director,3.0
Sales Representative,3.0
Developer,3.0
Sales Executive,2.0
Manufacturing Dir...,2.0
Laboratory Techni...,2.0
Research Scientist,2.0
Human Resources,2.0
Healthcare Repres...,2.0
Manager,2.0


### We know from the previous queries that "Support" team has highest attrition. The below query shows, the internal Support staff that do not travel att all have higher rate of attrition compared to the one's who may travel frequently or even rarely.

In [32]:
Attrition_BusinessTravel = spark.sql('''SELECT Department, BusinessTravel, COUNT(*) AS Total_Counts FROM HRA_Records
                                        WHERE Department = "Support" AND Attrition = "Yes"
                                        GROUP BY Department, BusinessTravel
                                        ORDER BY Total_Counts DESC''')

In [33]:
Attrition_BusinessTravel

Department,BusinessTravel,Total_Counts
Support,Non-Travel,139307
Support,Travel_Frequently,138978
Support,Travel_Rarely,138865


In [34]:
Attrition_Supp_JobLevel = spark.sql('''SELECT BusinessTravel, AVG(JobLevel) as JobLevel, COUNT(*) AS Total_Counts FROM HRA_Records
                                        WHERE Department = "Support" AND Attrition = "Yes"
                                        GROUP BY BusinessTravel
                                        ORDER BY Total_Counts DESC''')

### In this dataaset, JobLevel is between 1 to 4, where 1 being junior most to 4 being senior most. The query below shows, the attrition is majorly among the Senior level Support team employees who do not travel at all.

In [35]:
Attrition_Supp_JobLevel

BusinessTravel,JobLevel,Total_Counts
Non-Travel,3.008664316940283,139307
Travel_Frequently,3.0001942753529334,138978
Travel_Rarely,2.9930508047384152,138865


### The following query, gives us teh relationship between the employess that have left whether they worked overtime or not. And the max count of these employees DID NOT work overtime.

In [56]:
Attrition_OverTime = spark.sql('''SELECT OverTime, COUNT(*) as Total_Counts FROM HRA_Records
                                  WHERE Attrition = "Yes"
                                  GROUP BY OverTime
                                  ORDER BY Total_Counts DESC''')

In [57]:
Attrition_OverTime

OverTime,Total_Counts
No,1250862
Yes,1248664


### The below query shows, employees with highest number of training have highest attrition. This show's training and development doesn't necessarily help retaining employees

In [38]:
Attrition_TrainingTimesLastYear = spark.sql('''SELECT TrainingTimesLastYear, COUNT(*) as Total_Counts FROM HRA_Records
                                               WHERE Attrition = "Yes"
                                               GROUP BY TrainingTimesLastYear
                                               ORDER BY Total_Counts DESC''')

In [39]:
Attrition_TrainingTimesLastYear

TrainingTimesLastYear,Total_Counts
5,416883
2,416858
1,416785
6,416703
4,416625
3,415672


### The below query shows, employees with higher education have less number of attrition compared to the employees at level 1 education.

In [40]:
Attrition_Education = spark.sql('''SELECT Education, COUNT(*) as Total_Counts FROM HRA_Records
                                   WHERE Attrition = "Yes"
                                   GROUP BY Education
                                   ORDER BY Total_Counts DESC''')

In [41]:
Attrition_Education

Education,Total_Counts
1,500415
3,500162
2,499985
4,499715
5,499249


### The below query shows, the average hike is highest for "Research & Development" Department. Although other departments have a similar hike, "Research & Development" avg hikes are slightly higher.

In [42]:
Attrition_Department_Hike = spark.sql('''SELECT Department, AVG(PercentSalaryHike) as Total_Counts FROM HRA_Records
                                         GROUP BY Department
                                         ORDER BY Total_Counts DESC''')


In [43]:
Attrition_Department_Hike

Department,Total_Counts
Research & Develo...,24.519495835217874
Support,24.496678479980332
Human Resources,24.493827175311583
Sales,24.49217571429086
Software,24.49050959715114
Hardware,24.48624407517471


### The below job roles fall under department "Research & Development" and "Sales Executive" job role get's a higher hike percent compared to other roles

In [46]:
Attrition_JobRole_Hike = spark.sql('''SELECT JobRole, AVG(PercentSalaryHike) as Total_Counts FROM HRA_Records
                                      WHERE Department = 'Research & Development'
                                      GROUP BY JobRole
                                      ORDER BY Total_Counts DESC''')

In [47]:
Attrition_JobRole_Hike

JobRole,Total_Counts
Sales Executive,24.592842044925305
Human Resources,24.579978367984616
Healthcare Repres...,24.55573410723494
Research Director,24.540948662986338
Developer,24.534868169659916
Manager,24.53121690574757
Manufacturing Dir...,24.4751189235175
Sales Representative,24.4700189707754
Research Scientist,24.4696898637517
Laboratory Techni...,24.44375195891699
